### Scraping data from wikipedia

In [18]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

raw= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
bs = BeautifulSoup(raw,'xml')
bs.title

<title>List of postal codes of Canada: M - Wikipedia</title>

### Creating dataframe

In [10]:
table = bs.find('table')
PostalCode = []
Borough = []
Neighborhood = []

for tr_cell in table.find_all('tr'):
    counter = 1
    PostalCode_var = -1
    Borough_var = -1
    Neighborhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            PostalCode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
        if counter == 3: 
            Neighborhood_var = str(td_cell.text).strip()
            tag_a_Neighborhood = td_cell.find('a')
        counter +=1
        if (PostalCode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighborhood_var == 'Not assigned'):
            continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighborhood is None)):
            continue     
    except:
        pass
    if(PostalCode_var == -1 or Borough_var == -1 or Neighborhood_var == -1):
        continue  
        
    PostalCode.append(PostalCode_var)
    Borough.append(Borough_var)
    Neighborhood.append(Neighborhood_var)

In [12]:
uni_post = set(PostalCode)
#Number of unique postalcodes
print(len(uni_post))
PostalCode_u      = []
Borough_u       = []
Neighborhood_u = []


for post in uni_post:
    p_var = ''; b_var = ''; n_var = ''; 
    for post_idx, post_element in enumerate(PostalCode):
        if post == post_element:
            p_var = post_element;
            b_var = Borough[post_idx]
            if n_var == '': 
                n_var = Neighborhood[post_idx]
            else:
                n_var = n_var + ', ' + Neighborhood[post_idx]
    PostalCode_u.append(p_var)
    Borough_u.append(b_var)
    Neighborhood_u.append(n_var)

78


In [13]:
toronto = {'PostalCode':PostalCode_u, 'Borough':Borough_u, 'Neighborhood':Neighborhood_u}
df = pd.DataFrame.from_dict(toronto)
df.to_csv('Toronto.csv')
df.head()

,PostalCode,Borough,Neighborhood
0,M1V,Scarborough,"Agincourt North, Milliken"
1,M3L,North York,Downsview West
2,M6R,West Toronto,"Parkdale, Roncesvalles"
3,M5M,North York,Bedford Park
4,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market"


### Use the .shape method to print the number of rows of your dataframe

In [14]:
df.shape

(78, 3)

### Scraping geospatial data

In [19]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df1 = df.merge(coordinates, on="PostalCode", how="left")